In [0]:
# Avirut Mehta
# 200320@2030

In [0]:
import pandas as pd
import numpy as np

In [0]:
# source: https://wonder.cdc.gov/ucd-icd10.html
datadir = '/content/drive/My Drive/Global Health Project/Analysis/wonderraw/'
yearrange = range(2006, 2014+1)

In [0]:
def processWonderDF(path):
  raw = pd.read_csv(path, delim_whitespace=True)
  endind = raw.index[raw['Notes']=='Total']
  raw = raw.iloc[0:endind[0]]

  raw.loc[:, '%'] = raw.loc[:, 'Rate']
  raw.loc[:, 'Age'] = raw.loc[:, 'Crude']
  raw.loc[:, 'Crude'] = raw.loc[:, 'Population']
  raw.loc[:, 'Population'] = raw.loc[:, 'Deaths']
  raw.loc[:, 'Deaths'] = raw.loc[:, 'County Code']
  raw.loc[:, 'County'] = raw.loc[:, 'Notes']

  drops = ['Notes', 'County Code', 'Rate', 'Adjusted', 'Rate.1', 'of', 'Total', 'Deaths.1']
  raw = raw.drop(drops, axis='columns')

  raw = raw.rename(columns={'County': 'county', 'Deaths': 'deaths', 'Population': 'pop', 
                          'Crude': 'crudeDeathRate', 'Age': 'ageAdjDeathRate', '%': 'percentTotDeath'})
  for index, row in raw.iterrows():
    justCounty = row['county'].split(' County')[0]
    raw.at[index, 'county'] = justCounty
  return raw

In [0]:
allData = processWonderDF(datadir+'all.txt')
yearsData = {year : processWonderDF(datadir+str(year)+'.txt') for year in yearrange}

In [0]:
counties = allData['county'].tolist()
years = [year for year in yearrange]*len(counties)
counties = [[county]*len(yearrange) for county in counties]
counties = [item for sublist in counties for item in sublist]
dIndex = pd.MultiIndex.from_arrays([counties, years], names=['County', 'Year'])
cols = ['Suicides', 'PopCDC']
d = pd.DataFrame(np.zeros((len(counties), len(cols))), index=dIndex, columns=cols)

In [0]:
for year in yearsData:
  cdcData = yearsData[year]
  for index, row in cdcData.iterrows():
    county = row['county']
    d.loc[(county, year), 'Suicides'] = allData.at[index, 'deaths'] - cdcData.at[index, 'deaths']
    d.loc[(county, year), 'PopCDC'] = allData.at[index, 'pop'] - cdcData.at[index, 'pop']

In [0]:
d.to_csv('/content/drive/My Drive/Global Health Project/Analysis/wonder_sanitized_am_200320.csv')